In [1]:
""" is this script even needed? maybe there are weights already in the files """

import icecube
from icecube import icetray, dataio
from icecube import MuonGun
from I3Tray import I3Tray
import glob
import os

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

In [2]:
def harvest_generators(infiles):
    """
    Harvest serialized generator configurations from a set of I3 files.
    """
    from icecube.icetray.i3logging import log_info as log
    generator = None
    for fname in infiles:
        f = dataio.I3File(fname)
        fr = f.pop_frame(icetray.I3Frame.Stream('S'))
        f.close()
        if fr is not None:
            for k in fr.keys():
                v = fr[k]
                if isinstance(v, MuonGun.GenerationProbability):
                    log('%s: found "%s" (%s)' % (fname, k, type(v).__name__), unit="MuonGun")
                    if generator is None:
                        generator = v
                    else:
                        generator += v
    return generator

In [3]:
model = MuonGun.load_model('GaisserH4a_atmod12_SIBYLL')

# dataset enum
for enum in range(1,6):
    # which dataset
    if enum == 1:
        datapath = "/data/user/axelpo/LLP-data/DarkLeptonicScalar.mass-130.eps-5e-6.nevents-5000_230919.196721189/"
        name     = "DLS"
        folder_path = "analyze_"+datapath[27:-1]+"/"
    elif enum == 2:
        datapath = "/data/user/axelpo/LLP-data/DarkLeptonicScalar.mass-130.eps-5e-6.nevents-5000_naturalrate_230919.196721196/"
        name     = "DLS_naturalrate"
        folder_path = "analyze_"+datapath[27:-1]+"/"
    elif enum == 3:
        datapath = "/data/user/axelpo/LLP-data/NO_LLP.DarkLeptonicScalar.mass-130.eps-5e-6.nevents-5000_230919.196721202/"
        name     = "NO_LLP"
        folder_path = "analyze_"+datapath[27:-1]+"/"
    elif enum == 4:
        datapath = "/data/user/axelpo/LLP-data/NO_LLP.DarkLeptonicScalar.mass-130.eps-5e-6.nevents-5000_naturalrate_230919.196721201/"
        name     = "NO_LLP_naturalrate"
        folder_path = "analyze_"+datapath[27:-1]+"/"
    elif enum == 5:
        datapath = "/data/user/axelpo/LLP-data/DarkLeptonicScalar.mass-130.eps-5e-6.nevents-5000_eitherdecayorprod_230921.196832696/"
        name     = "DLS_eitherdecayorprod"
        folder_path = "analyze_"+datapath[27:-1]+"/"


    infiles = list(glob.glob(datapath+"LLPSim*/*.gz"))
    generator = harvest_generators(infiles)
    
    tray = I3Tray()
    tray.Add("I3Reader", filenamelist=infiles)
    tray.AddModule('I3MuonGun::WeightCalculatorModule', 'MuonWeight', Model=model,
        Generator=generator)
    tray.Execute()

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


In [7]:
#infiles = list(glob.glob(datapath+"LLPSim*/*.gz"))
model = MuonGun.load_model('GaisserH4a_atmod12_SIBYLL')
infiles = ["/data/user/axelpo/LLP-data/NO_LLP.DarkLeptonicScalar.mass-130.eps-5e-6.nevents-5000_230919.196721202/L2.i3.gz"]

generator = harvest_generators(infiles)

tray = I3Tray()
tray.Add("I3Reader", filenamelist=infiles)
tray.AddModule('I3MuonGun::WeightCalculatorModule', 'MuonWeight', Model=model,
    Generator=generator)
tray.Add("I3Writer", filename="L2_weighted.i3.gz")
tray.Execute()

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
